In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers import Convolution1D, Dense, Dropout, MaxPooling1D, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from keras import callbacks
from keras.callbacks import CSVLogger
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler 
import pickle
from sklearn import metrics
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas  as pd
import numpy as np
import os
import time
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
        f1_score, classification_report, 
        confusion_matrix, roc_curve, 
        roc_auc_score, accuracy_score,
        log_loss)
from sklearn import __version__ as sklearn_version
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import NearMiss
from datetime import datetime
np.random.seed(0)

In [ ]:
dir2='/content/drive/MyDrive/botnet_dataset/device2/'
device2_df=pd.DataFrame()
i=0
for file in os.listdir(dir2+'gafgyt/'):
  df_file = pd.read_csv(os.path.join(dir2+'gafgyt/', file), encoding = 'ISO-8859-1',low_memory=False)
  df_file["label"]=i
  i=i+1
  device2_df=pd.concat([device2_df, df_file])
for file in os.listdir(dir2+'mirai/'):
  df_file = pd.read_csv(os.path.join(dir2+'mirai/', file), encoding = 'ISO-8859-1',low_memory=False)
  df_file["label"]=i
  i=i+1
  device2_df=pd.concat([device2_df, df_file])
df_file=pd.read_csv('/content/drive/MyDrive/botnet_dataset/device2/benign.csv')
df_file['label']=i
device2_df=pd.concat([device2_df, df_file])

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(device2_df, test_size=0.2)


In [ ]:
X_train=train.drop(['label'],axis=1)
y_train=train['label']


In [ ]:
from sklearn.tree import DecisionTreeClassifier
model_lr=DecisionTreeClassifier()
rfe = RFE(model_lr, n_features_to_select=50,step=4)
rfe.fit(X_train,y_train)

RFE(estimator=DecisionTreeClassifier(), n_features_to_select=50, step=4)

In [ ]:
from operator import itemgetter
features = X_train.columns.to_list()
i=0
feat_arr_rem=[]
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    if(x!=1):
      i=i+1
      feat_arr_rem.append(y)

In [ ]:
mod_df=device2_df.copy()
for str in feat_arr_rem:
  if(str=='label'):
    continue
  mod_df.drop([str],axis=1, inplace=True)
mod_df.head()

,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,MI_dir_L0.1_mean,MI_dir_L0.1_variance,MI_dir_L0.01_weight,MI_dir_L0.01_mean,MI_dir_L0.01_variance,H_L5_weight,...,HpHp_L1_pcc,HpHp_L0.1_weight,HpHp_L0.1_mean,HpHp_L0.1_std,HpHp_L0.1_magnitude,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,label
0,1.000000,98.000000,0.000000e+00,1.000000,98.000000,0.000000e+00,1.000000,98.000000,0.000000,1.000000,...,0.0,1.00000,98.0,0.000000,98.000000,0.000000e+00,0.0,0.0,1.000000,0
1,1.492583,98.000000,3.637979e-12,1.931640,98.000000,1.818989e-12,1.992944,98.000000,0.000000,1.029000,...,0.0,1.93164,98.0,0.000001,138.592929,1.818989e-12,0.0,0.0,1.992944,0
2,2.294102,84.051188,2.517926e+02,2.904273,86.981750,2.311822e+02,2.990102,87.298025,227.930928,1.504156,...,0.0,1.00000,66.0,0.000000,114.856432,0.000000e+00,0.0,0.0,1.000000,0
3,3.280499,80.987267,1.964467e+02,3.902546,83.655268,2.040614e+02,3.989925,83.965124,204.017078,2.460087,...,0.0,1.00000,74.0,0.000000,74.000000,0.000000e+00,0.0,0.0,1.000000,0
4,4.280490,79.354915,1.592943e+02,4.902545,81.685828,1.775746e+02,4.989924,81.968075,179.043909,3.460055,...,0.0,1.00000,74.0,0.000000,74.000000,0.000000e+00,0.0,0.0,1.000000,0


In [ ]:
mod_df.to_csv('/content/drive/MyDrive/botnet_dataset/device2/dataset_final.csv')

In [ ]:
x=mod_df.drop(['label'],axis=1)
y=mod_df['label']
normalized_df=(x-x.mean())/x.std()
y.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(normalized_df, y, test_size=0.2, random_state=0)
y_train = np.array(y_train)

y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1]))
print('xtrain={}, ytrain={}, xtest={}, ytest={}'.format(X_train.shape,y_train.shape,X_test.shape,y_test.shape))

xtrain=(668700, 65), ytrain=(668700,), xtest=(167176, 65), ytest=(167176,)


In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_train.shape

(668700,)

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score
print('XGBoost Classifier')

print('Accuracy = ', metrics.accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                 digits=2, 
                                                                 output_dict=False))

print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

XGBoost Classifier
Accuracy =  88.6742116093219
Confusion Matrix =
 [[10741     1     1     0     0     0     0     0     0     0     0]
 [    5  5931     0     1     0     0     0     0     0     0     0]
 [    1     0  5541     0     0     0     0     0     1     0     0]
 [    0     0     0    14 18882     0     0     0     0     1     2]
 [    1     0     0     2 20947     0     0     0     0     0     1]
 [    0     0     0     0     0 22725     0     0     0     0     0]
 [    0     0     0     0     0     0  8620     0     0     0     0]
 [    0     0     0     0     0     0     0 23492     0     0     0]
 [    0     0     0     0     0     0     2     0 30261     7     0]
 [    0     0     0     0     0     0     0     0    24 17331     1]
 [    0     0     0     0     1     0     0     0     0     0  2639]]
Recall = 0.8867421160932191
Classification Report =
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     10743
          

In [ ]:
def rf_classifier(data, device_name, scaling=False,):
    X = data.drop(['label'], axis=1)
    y = data['label']
    print(f'Original Shape:{X.shape}, {y.shape}')
    if scaling == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        model_name = f'{device_name}_without_scaling_unbalanced_model.pkl'
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        model_name = f'{device_name}_with_scaling_unbalanced_model.pkl'
    
    clf = RandomForestClassifier()
    start=time.time()
    model_res = clf.fit(X_train, y_train)
    end=time.time()
    tt=end-start
    y_pred = model_res.predict(X_test)
    y_pred_prob = model_res.predict_proba(X_test)
    lr_probs = y_pred_prob[:,1]
    ac = accuracy_score(y_test, y_pred)
    importances = pd.DataFrame({'feature':X.columns,'importance':np.round(clf.feature_importances_,3)})
    importances = importances.sort_values('importance',ascending=False).set_index('feature')

    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    print("Random Forest Classifier")
    print(f"Training time: {tt}s")
    print('Accuracy=%.3f' % (ac))

    print('f1-score=%.3f' % (f1))
    print('Confusion Matrix = \n', cm)
    print('Classification Report = \n',classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier(mod_df,'device1',False)

Original Shape:(835876, 65), (835876,)
Random Forest Classifier
Training time: 269.48349237442017s
Accuracy=0.887
f1-score=0.848
Confusion Matrix = 
 [[15992     2     4     1     0     0     0     0     0     0     0]
 [    1  9069     1     1     0     0     0     0     0     0     0]
 [    0     0  8272     1     0     0     1     0     0     0     0]
 [    0     1     1    20 28324     0     0     0     0     0     2]
 [    0     1     1     2 31127     0     0     0     0     0     1]
 [    0     0     0     0     0 34100     0     0     0     1     0]
 [    0     0     0     0     1     0 13078     0     3     0     0]
 [    0     0     0     0     0     0     0 35079     0     0     0]
 [    0     0     0     0     0     0     1     0 45510    10     0]
 [    0     0     0     0     0     0     0     0    11 26202     0]
 [    0     0     0     0     2     0     0     0     0     1  3939]]
Classification Report = 
               precision    recall  f1-score   support

         

In [ ]:
rf_classifier(mod_df,'device1',True)

Original Shape:(835876, 65), (835876,)
Random Forest Classifier
Training time: 241.20975732803345s
Accuracy=0.887
f1-score=0.848
Confusion Matrix = 
 [[15994     1     3     1     0     0     0     0     0     0     0]
 [    3  9067     1     1     0     0     0     0     0     0     0]
 [    0     0  8272     1     0     0     1     0     0     0     0]
 [    0     1     1    20 28324     0     0     0     0     0     2]
 [    0     1     1     2 31127     0     0     0     0     0     1]
 [    0     0     0     0     0 34101     0     0     0     0     0]
 [    0     0     0     0     1     0 13078     0     3     0     0]
 [    0     0     0     0     0     0     0 35079     0     0     0]
 [    0     0     0     0     0     0     1     0 45513     7     0]
 [    0     0     0     0     0     0     0     0    11 26202     0]
 [    0     0     0     0     2     0     0     0     0     1  3939]]
Classification Report = 
               precision    recall  f1-score   support

         

In [13]:
from sklearn import metrics
from sklearn.metrics import f1_score
def knn_classifier_unbalanced(data, scaling=False):
    X = data.drop(['label'], axis=1)
    y = data['label']
    print(f'Original Shape:{X.shape}, {y.shape}')
    test_scores = []
    train_scores = []
    error = []
    if scaling == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
    knn = KNeighborsClassifier(3)
    start=time.time()
    knn.fit(X_train,y_train)
    end=time.time()
    tt=end-start
    y_pred = knn.predict(X_test)
    error.append(np.mean(y_pred != y_test))
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))
    print("KNN")
    print(f"Training time: {tt}s")
    print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
    print('Classification Report = \n',classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(f'Accuracy on Training set:{train_scores}')
    print(f'Accuracy on Test set:{test_scores}')

In [14]:
temp_df=pd.read_csv('/content/drive/MyDrive/botnet_dataset/device2/dataset_final.csv')

In [15]:
knn_classifier_unbalanced(temp_df,False)

Original Shape:(835876, 51), (835876,)
KNN
Training time: 1.3853652477264404s
Recall = 0.6172720855947648
Classification Report = 
               precision    recall  f1-score   support

           0       0.05      0.12      0.07     15999
           1       0.07      0.09      0.08      9072
           2       0.98      0.98      0.98      8274
           3       1.00      1.00      1.00     28348
           4       1.00      1.00      1.00     31132
           5       0.36      0.46      0.40     34101
           6       1.00      1.00      1.00     13082
           7       0.59      0.47      0.53     35079
           8       0.77      0.54      0.63     45521
           9       0.82      0.42      0.55     26213
          10       1.00      0.98      0.99      3942

    accuracy                           0.62    250763
   macro avg       0.69      0.64      0.66    250763
weighted avg       0.70      0.62      0.65    250763

[[ 1896  5476    33     4     3  5972     8  2215   329

In [16]:
knn_classifier_unbalanced(temp_df,True)

Original Shape:(835876, 51), (835876,)
KNN
Training time: 0.22026443481445312s
Recall = 0.9055921328106619
Classification Report = 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     15999
           1       1.00      1.00      1.00      9072
           2       1.00      1.00      1.00      8274
           3       0.58      0.58      0.58     28348
           4       0.62      0.62      0.62     31132
           5       1.00      1.00      1.00     34101
           6       1.00      1.00      1.00     13082
           7       1.00      1.00      1.00     35079
           8       1.00      1.00      1.00     45521
           9       1.00      1.00      1.00     26213
          10       1.00      1.00      1.00      3942

    accuracy                           0.91    250763
   macro avg       0.93      0.93      0.93    250763
weighted avg       0.91      0.91      0.91    250763

[[15981    11     3     0     0     0     0     4     

In [18]:
def dt_classifier(data, device_name, scaling=False,):
    X = data.drop(['label'], axis=1)
    y = data['label']
    print(f'Original Shape:{X.shape}, {y.shape}')
    if scaling == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        model_name = f'{device_name}_without_scaling_unbalanced_model_decision_tree_gini.pkl'
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        model_name = f'{device_name}_with_scaling_unbalanced_model_decision_tree_gini.pkl'
    
    clf = DecisionTreeClassifier(criterion='gini', random_state= 1234, max_depth  = 5 )
    start=time.time()
    model_res = clf.fit(X_train, y_train)
    end=time.time()
    tt=end-start
    y_pred = model_res.predict(X_test)
    y_pred_prob = model_res.predict_proba(X_test)
    lr_probs = y_pred_prob[:,1]
    ac = accuracy_score(y_test, y_pred)
    importances = pd.DataFrame({'feature':X.columns,'importance':np.round(clf.feature_importances_,3)})
    importances = importances.sort_values('importance',ascending=False).set_index('feature')
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    print('Decision tree')
    print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
    print(f"Training time: {tt}s")
    print('Accuracy=%.3f' % (ac))
    print('f1-score=%.3f' % (f1))
    print('Confusion Matrix = \n', cm)
    print('Classification Report',classification_report(y_test, y_pred))

In [20]:
dt_classifier(temp_df,'device2',True)

Original Shape:(835876, 51), (835876,)
Decision tree
Recall = 0.8346965062628857
Training time: 27.45733904838562s
Accuracy=0.835
f1-score=0.775
Confusion Matrix = 
 [[15995     0     4     0     0     0     0     0     0     0     0]
 [ 9068     0     4     0     0     0     0     0     0     0     0]
 [    0     0  8274     0     0     0     0     0     0     0     0]
 [    0     0    24     0 28324     0     0     0     0     0     0]
 [    0     0    21     0 31108     0     3     0     0     0     0]
 [    0     0     0     0     0 34101     0     0     0     0     0]
 [    1     0     3     0     0     0 13076     2     0     0     0]
 [    0     0     0     0     0     0     0 35079     0     0     0]
 [    0     0    56     0     0     0     2     0 45463     0     0]
 [    0     0     0     0     0     0     0     0     0 26213     0]
 [    0     0  3939     0     0     0     1     0     0     0     2]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report               precision    recall  f1-score   support

           0       0.64      1.00      0.78     15999
           1       0.00      0.00      0.00      9072
           2       0.67      1.00      0.80      8274
           3       0.00      0.00      0.00     28348
           4       0.52      1.00      0.69     31132
           5       1.00      1.00      1.00     34101
           6       1.00      1.00      1.00     13082
           7       1.00      1.00      1.00     35079
           8       1.00      1.00      1.00     45521
           9       1.00      1.00      1.00     26213
          10       1.00      0.00      0.00      3942

    accuracy                           0.83    250763
   macro avg       0.71      0.73      0.66    250763
weighted avg       0.76      0.83      0.78    250763



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
dt_classifier(temp_df,'device2',False)

Original Shape:(835876, 51), (835876,)
Decision tree
Recall = 0.9456419009183971
Training time: 22.391864776611328s
Accuracy=0.946
f1-score=0.925
Confusion Matrix = 
 [[15995     0     4     0     0     0     0     0     0     0     0]
 [ 9068     0     4     0     0     0     0     0     0     0     0]
 [    0     0  8274     0     0     0     0     0     0     0     0]
 [    0     0    24 28324     0     0     0     0     0     0     0]
 [    0     0    21   503 30605     0     3     0     0     0     0]
 [    0     0     0     0     0 34101     0     0     0     0     0]
 [    0     0     3     0     0     0 13076     2     1     0     0]
 [    0     0     0     0     0     0     0 35079     0     0     0]
 [    0     0    56     0     0     0     2     0 45463     0     0]
 [    0     0     0     0     0     0     0     0     0 26213     0]
 [    0     0  3939     0     0     0     1     0     0     0     2]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report               precision    recall  f1-score   support

           0       0.64      1.00      0.78     15999
           1       0.00      0.00      0.00      9072
           2       0.67      1.00      0.80      8274
           3       0.98      1.00      0.99     28348
           4       1.00      0.98      0.99     31132
           5       1.00      1.00      1.00     34101
           6       1.00      1.00      1.00     13082
           7       1.00      1.00      1.00     35079
           8       1.00      1.00      1.00     45521
           9       1.00      1.00      1.00     26213
          10       1.00      0.00      0.00      3942

    accuracy                           0.95    250763
   macro avg       0.84      0.82      0.78    250763
weighted avg       0.93      0.95      0.93    250763



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
